In [3]:
import json
import numpy as np
import tensorflow_datasets as tfds
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import mediapy
from scipy.spatial.transform import Rotation as R

In [2]:
pip install mediapy

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Load the DROID subset dataset
ds = tfds.load("droid_100", data_dir="gs://gresearch/robotics", split="train")

2025-11-03 18:21:21.973557: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-03 18:21:21.991370: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-03 18:21:22.524213: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-03 18:21:24.156966: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

In [8]:
from pathlib import Path
import os
# Automatically detect the current notebook directory
path_to_droid_repo = str(Path(__file__).parent) if '__file__' in globals() else str(Path.cwd())

# Load the extrinsics
cam2base_extrinsics_path = f"{path_to_droid_repo}/cam2base_extrinsics.json"
with open(cam2base_extrinsics_path, "r") as f:
    cam2base_extrinsics = json.load(f)

# Load the intrinsics
intrinsics_path = f"{path_to_droid_repo}/intrinsics.json"
with open(intrinsics_path, "r") as f:
    intrinsics = json.load(f)

# Load mapping from episode ID to path, then invert
episode_id_to_path_path = f"{path_to_droid_repo}/episode_id_to_path.json"
with open(episode_id_to_path_path, "r") as f:
    episode_id_to_path = json.load(f)
episode_path_to_id = {v: k for k, v in episode_id_to_path.items()}

# Load camera serials
camera_serials_path = f"{path_to_droid_repo}/camera_serials.json"
with open(camera_serials_path, "r") as f:
    camera_serials = json.load(f)

In [9]:
# Iterate through the dataset to find the first episode that has a cam2base extrinsics entry
eps = []
for ep in tqdm(ds):
    file_path = ep["episode_metadata"]["file_path"].numpy().decode("utf-8")
    recording_folderpath = ep["episode_metadata"]["recording_folderpath"].numpy().decode("utf-8")

    episode_path = file_path.split("r2d2-data-full/")[1].split("/trajectory")[0]
    if episode_path not in episode_path_to_id:
        continue
    episode_id = episode_path_to_id[episode_path]
    
    if episode_id in cam2base_extrinsics:
        eps.append(ep)
    
    if len(eps) >= 10:
        break

 25%|██▌       | 25/100 [00:58<02:54,  2.33s/it]


In [10]:
# Iterate through the extrinsics to find key that is a digit
# This is the camera serial number, and the corresponding value is the extrinsics
for k, v in cam2base_extrinsics[episode_id].items():
    if k.isdigit():
        camera_serial = k
        extracted_extrinsics = v
        break

# Also lets us get the intrinsics
extracted_intrinsics = intrinsics[episode_id][camera_serial]

# Using the camera serial, find the corresponding camera name (which is used to determine
# which image stream in the episode to use)
camera_serials_to_name = {v: k for k, v in camera_serials[episode_id].items()}
calib_camera_name = camera_serials_to_name[camera_serial]

if calib_camera_name == "ext1_cam_serial":
    calib_image_name = "exterior_image_1_left"
elif calib_camera_name == "ext2_cam_serial":
    calib_image_name = "exterior_image_2_left"
else:
    raise ValueError(f"Unknown camera name: {calib_camera_name}")

print(f"Camera with calibration data: {calib_camera_name} --> {calib_image_name}")

Camera with calibration data: ext1_cam_serial --> exterior_image_1_left


In [11]:
# Convert the extrinsics to a homogeneous transformation matrix
pos = extracted_extrinsics[0:3] # translation
rot_mat = R.from_euler("xyz", extracted_extrinsics[3:6]).as_matrix() # rotation

# Make homogenous transformation matrix
cam_to_base_extrinsics_matrix = np.eye(4)
cam_to_base_extrinsics_matrix[:3, :3] = rot_mat
cam_to_base_extrinsics_matrix[:3, 3] = pos

print(cam_to_base_extrinsics_matrix)

[[-0.79138441 -0.21481151  0.57233445  0.27362273]
 [-0.61099013  0.30863306 -0.72899705  0.45935   ]
 [-0.02004438 -0.9266076  -0.37549511  0.50548118]
 [ 0.          0.          0.          1.        ]]


In [12]:
# Convert the intrinsics to a matrix
fx, cx, fy, cy = extracted_intrinsics["cameraMatrix"]
intrinsics_matrix = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0, 0, 1]
])
print(intrinsics_matrix)

[[522.96893311   0.         634.02514648]
 [  0.         522.96893311 365.70669556]
 [  0.           0.           1.        ]]


In [13]:
# Save all observations for the calibrated camera and corresponding gripper positions
images = []
cartesian_poses = []
for step in ep["steps"]:
    image = step["observation"][calib_image_name].numpy()
    images.append(image)
    cartesian_pose = step["observation"]["cartesian_position"].numpy()
    cartesian_poses.append(cartesian_pose)

# length images x 6
cartesian_poses = np.array(cartesian_poses)
# Remove the rotation and make homogeneous: --> length images x 3 --> length images x 4
cartesian_homogeneous_positions = cartesian_poses[:, :3]
cartesian_homogeneous_positions = np.hstack(
    (cartesian_homogeneous_positions, np.ones((cartesian_homogeneous_positions.shape[0], 1)))
)

# Transpose to support matrix multiplication: --> 4 x length images
gripper_position_base = cartesian_homogeneous_positions.T

2025-11-03 18:33:03.133793: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [14]:
# Transform gripper position to camera frame, then remove homogeneous component
base_to_cam_extrinsics_matrix = np.linalg.inv(cam_to_base_extrinsics_matrix)
robot_gripper_position_cam = base_to_cam_extrinsics_matrix @ gripper_position_base
robot_gripper_position_cam = robot_gripper_position_cam[:3] # Now 3 x length images

In [15]:
# Finally, use intrinsics to project the gripper position in camera frame into pixel space
pixel_positions = intrinsics_matrix @ robot_gripper_position_cam[:3]
pixel_positions = pixel_positions[:2] / pixel_positions[2]

In [17]:
# Visualize!
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend for saving
import matplotlib.pyplot as plt
vis_images = []
temp_img_path = f"{path_to_droid_repo}/TEMP.png"

for i, image in enumerate(tqdm(images)):
    if i % 10 != 0:
        continue
    
    fig, axs = plt.subplots(1, 1)  # Use explicit arguments
    x, y = pixel_positions[0, i] / 1280 * 320, pixel_positions[1, i] / 720 * 180 # Scale to match image dimensions

    # clip coords
    x = np.clip(x, 0, 320)
    y = np.clip(y, 0, 180)

    axs.imshow(image)
    axs.scatter(x, y, c='red', s=20)
    axs.set_xlim(0, 320)
    axs.set_ylim(180, 0)  # Invert y-axis to match image

    # turn off axes
    axs.axis('off')

    # save the figure, then reopen it as PIL image
    plt.savefig(temp_img_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    vis_image = Image.open(temp_img_path).convert("RGB")
    vis_images.append(np.array(vis_image))


100%|██████████| 558/558 [00:03<00:00, 147.44it/s]


In [20]:
# Visualize the video
mediapy.show_video(
    vis_images,
    fps=8
)

This browser does not support the video tag.
